In [2]:
import transformers

print(transformers.__version__)

/Users/dz/.pyenv/versions/3.10.14/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


4.43.1


In [4]:
from datasets import load_dataset
datasets = load_dataset('wikitext', 'wikitext-2-raw-v1')

Generating validation split: 100%|████████████████████████████████████████████████████████████████████████████████| 3760/3760 [00:00<00:00, 1126792.16 examples/s]


In [5]:
datasets["train"][10]

{'text': ' The game \'s battle system , the BliTZ system , is carried over directly from Valkyira Chronicles . During missions , players select each unit using a top @-@ down perspective of the battlefield map : once a character is selected , the player moves the character around the battlefield in third @-@ person . A character can only act once per @-@ turn , but characters can be granted multiple turns at the expense of other characters \' turns . Each character has a field and distance of movement limited by their Action Gauge . Up to nine characters can be assigned to a single mission . During gameplay , characters will call out if something happens to them , such as their health points ( HP ) getting low or being knocked out by enemy attacks . Each character has specific " Potentials " , skills unique to each character . They are divided into " Personal Potential " , which are innate skills that remain unaltered unless otherwise dictated by the story and can either help or impede

In [6]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))
show_random_elements(datasets["train"])

,text
0,"The two most commonly used classification schemata for AML are the older French @-@ American @-@ British ( FAB ) system and the newer World Health Organization ( WHO ) system . According to the widely used WHO criteria , the diagnosis of AML is established by demonstrating involvement of more than 20 % of the blood and / or bone marrow by leukemic myeloblasts , except in the three best prognosis forms of AML with recurrent genetic abnormalities ( t ( 8 ; 21 ) , inv ( 16 ) , and t ( 15 ; 17 ) ) in which the presence of the genetic abnormality is diagnostic irrespective of blast percent . The French – American – British ( FAB ) classification is a bit more stringent , requiring a blast percentage of at least 30 % in bone marrow ( BM ) or peripheral blood ( PB ) for the diagnosis of AML . AML must be carefully differentiated from "" preleukemic "" conditions such as myelodysplastic or myeloproliferative syndromes , which are treated differently . \n"
1,"Wheeler had been keen to return to excavation in Britain . Based on the one he had organised in India , Wheeler developed an archaeological training course , which he ran at Verulamium in the summer of 1949 to instruct British students in the methodologies of excavation . In summer 1950 , he was invited by the Royal Commission on Historical Monuments to direct a trial excavation at Bindon Hill in Dorset . It was a leisurely project which he treated as a seaside holiday . He was invited by the Ancient Monuments Department of the Ministry of Works to excavate the Stanwick Iron Age Fortifications in North Riding , Yorkshire , which he proceeded to do over the summers of 1951 and 1952 . Aided by many old friends and colleagues from within the British archaeological scene , he was joined by Alcock and Alcock 's wife , among others . Wheeler published his report on the site in 1954 . \n"
2,= = = Club career = = = \n
3,= = = Strike = = = \n
4,"The fort remained as headquarters of South Australia 's permanent military force until the 1890s and as late as 1895 there were still plans to upgrade the 64 @-@ pounder armament , though without result . By 1901 the fort was manned on a caretaker basis only and no permanent forces were stationed . The Federal Government assumed responsibility for South Australian defence in 1903 and took over the fort . Though Glanville by then had no significant defence role , the state received £ 14 @,@ 739 in compensation . From that point its significance was not defence related but as the first and best preserved 19th @-@ century fortification in South Australia . \n"
5,
6,"The exterior scenes of the Howard Johnson 's hotel were filmed in October 2011 at the Regency Inn and Suites in Baldwin Park , California . The hotel operated as a Howard Johnson 's from 1967 until it was sold in 1995 . \n"
7,"Strings recorded at Ocean Way in Nashville , Tennessee . \n"
8,
9,"Ireland has nine universities , seven in the Republic of Ireland and two in Northern Ireland , including Trinity College , Dublin and the University College Dublin , as well as numerous third @-@ level colleges and institutes and a branch of the Open University , the Open University in Ireland . \n"


In [7]:
model_checkpoint = "gpt2"
tokenizer_checkpoint = "sgugger/gpt2-like-tokenizer"

from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint)

def tokenize_function(examples):
    return tokenizer(examples["text"])

tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

Map (num_proc=4): 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 3760/3760 [00:00<00:00, 19127.87 examples/s]


In [8]:
tokenized_datasets

DatasetDict({
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 3760
    })
})

In [18]:
tokenizer.vocab_size

25000

In [19]:
from transformers import AutoConfig, AutoModelForCausalLM

config = AutoConfig.from_pretrained(model_checkpoint)
model = AutoModelForCausalLM.from_config(config)

In [20]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [21]:
model_checkpoint = "bert-base-cased"
tokenizer_checkpoint = "sgugger/bert-like-tokenizer"
from transformers import AutoConfig, AutoModelForMaskedLM

config = AutoConfig.from_pretrained(model_checkpoint)
model = AutoModelForMaskedLM.from_config(config)
model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi